# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [192]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [193]:
!ls

env_variable.json	 QuanNguyen-b8bebb09be47.json  response_user_list.json
home_assignment_3.ipynb  README.md


In [194]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    #print(json_data)

In [195]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 

In [198]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

In [256]:
#user_dat

### TODO #1
Hoàn tất đoạn code sau

In [18]:
# Solution 1

## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [19]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [188]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


In [ ]:
a = list(map(str, input().rstrip().split()))#id name deleted profile is_admin is_bot tz updated 
b = list(map(str, input().rstrip().split()))#title phone skype real_name_normalized display_name_normalized

#### Tạo Function sử dụng các parameters bên ngoài get data từ chuối JSON User Slack

- Tham số a: dùng để get các thông tin keys mà value là giá trị chỉ có một dimension như là :String, number,..
- Tham số b: dùng để get các thông tin keys là value là giá trị có dimensions như là: list, dictionary.
- Flow control của function:
1. Tạo một dictionary (D1) chỉ lấy những para của tham số a từ original dict.
2. Tạo một dictionary (D2) khác để lấy nhứng subdata trong các keys của dict tạo từ B1 với các para từ tham số b.
3. Đối với các keys có values là dict hay list trong (D1) sẽ kiểm tra keys đó có trong tham số b không?. Nếu có sẽ get về, còn không thì gán giá trị D1.value() = D2.value()

#### Lưu ý:
- Vì length của original dict lớn hơn 1 nên sẽ thực hiện vòng lặp. Vì vậy D2 từ vòng lặp 2 trở đi các giá trị ở (D2) phải được append vào keys đã có ở vòng lặp 1. Vì dict không cho trùng key nên sẽ báo lỗi.
- Vì dữ liệu ít nên có thể sử dụng function dưới, khi dữ liệu quá nhiều sẽ tốn tài nguyên và thời gian cần optimize nếu gặp trường hợp đó. Nếu ai có ý tưởng optimize có thể liên hệ Quân Nguyễn trong Slack của DATACracy nhé. Cảm ơn.


In [84]:
## Solution 2
def get_user_json(data_farme,a,b):
    for i in range(len(data_farme)):
        user = {}
        for l in a:
            user[l] = data_farme[i][l]

        if i == 0:
            test ={}
            for j in user.keys():
                if type(user[j]) == dict:
                    z = [l for l in user[j].keys()]
                    for k in b:
                        if k in z:
                            test[k] = [user[j][k]]
                elif type(user[j]) == list:
                    z = [l for l in user[j][len(user[j])-1].keys()]
                    for k in z:
                        test[k] = [user[j][len(user[j])-1][k]]
                else:
                    test[j] = [user[j]]
        else:
            for j in user.keys():
                if type(user[j]) == dict:
                    z = [l for l in user[j].keys()]
                    for k in b:
                        if k in z:
                            test[k].append(user[j][k])
                elif type(user[j]) == list:
                    z = [l for l in user[j][len(user[j])-1].keys()]
                    for k in z:
                        test[k].append(user[j][len(user[j])-1][k])
                else:
                    test[j].append(user[j])
                    
    df_user = pd.DataFrame(test)
    return df_user
        
    

In [68]:
df_user = get_user_json(a,b)

In [63]:
process_time['updated_time_1'] = process_time['updated'].apply(lambda x : pd.Timestamp(x,unit='s'))

In [71]:
df_user = pd.merge(df_user, process_time[['id','updated_time_1']], how="left", on=["id"])

In [72]:
df_user

,id,name,deleted,title,phone,skype,real_name_normalized,display_name_normalized,is_admin,is_bot,tz,updated,updated_time_1
0,USLACKBOT,slackbot,False,,,,Slackbot,Slackbot,False,False,America/Los_Angeles,0,NaT
1,U01AT4T75JB,loclexuan26392,False,,,,Loc Le Xuan,Loc Le Xuan,False,False,Asia/Bangkok,1600924572,2020-09-24 05:16:12
2,U01AVDY7JET,locle.ds,False,,,,Loc Le Xuan,Loc Le Xuan,False,False,Asia/Bangkok,1600943506,2020-09-24 10:31:46
3,U01BE2PR6LU,maianhdang.ftu,False,Technical Contents,,,Dặng Huỳnh Mai Anh,MAD,True,False,Asia/Bangkok,1618595000,2021-04-16 17:43:20
4,U01C48T7S1J,huyenhoang.design,False,,,,Thanh Huyen Hoang,Thanh Huyen Hoang,False,False,Europe/Amsterdam,1601879503,2020-10-05 06:31:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,U020P6WPAQJ,thuynh.386,False,,,,Thuy Nguyen,,False,False,Asia/Bangkok,1620009844,2021-05-03 02:44:04
63,U020P877V88,huongntn2202,False,,,,Huong Nguyen Thi Nhi,,False,False,Asia/Bangkok,1619409574,2021-04-26 03:59:34
64,U021TFMH1RU,mquang.151999,False,,,,Quang Nguyen,Quang Nguyen,False,False,Asia/Bangkok,1621046249,2021-05-15 02:37:29
65,U021Y57RMN1,lamlv723,False,,,,Lam Le,Lam Le,False,False,Asia/Bangkok,1621134074,2021-05-16 03:01:14


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [73]:
!ls

env_variable.json	 QuanNguyen-b8bebb09be47.json  response_user_list.json
home_assignment_3.ipynb  README.md


In [243]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'QuanNguyen-b8bebb09be47.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [78]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1mQJRxLHzDPwpP_BAaCdtQGnP_f9ItcNg_dWaMmKs5eA' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, df_user) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

#### Link Google Sheet:
https://docs.google.com/spreadsheets/d/1mQJRxLHzDPwpP_BAaCdtQGnP_f9ItcNg_dWaMmKs5eA/edit#gid=0

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [79]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [80]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 63}

In [87]:
atom_assignment = get_user_json(channel_ls,['id','name_normalized','purpose','num_members'],['creator','last_set'])

### TODO#3 
* Tìm id của channel #atom-assignment2

In [107]:
atom_assignment[atom_assignment.name_normalized == 'atom-assignment2']

,id,name_normalized,creator,last_set,num_members
9,C021FSDN7LJ,atom-assignment2,U01BE2PR6LU,1621074519,38


### Slack API: List messages trong 1 channel

In [5]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [6]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [133]:
msg_ls[6]

{'client_msg_id': '811e1cc8-9a7a-4e2d-81dc-906eae8e8f3a',
 'type': 'message',
 'text': 'Chào mọi ngườii,\nem gửi assignment 2 ạ:\n<https://github.com/tinyvyy/atom-assignments/blob/main/ATOM_Home_Assignment_2.ipynb>',
 'user': 'U01UMBZHU2W',
 'ts': '1621044778.004500',
 'team': 'T01B7SGGMLJ',
 'blocks': [{'type': 'rich_text',
   'block_id': 'qjR',
   'elements': [{'type': 'rich_text_section',
     'elements': [{'type': 'text',
       'text': 'Chào mọi ngườii,\nem gửi assignment 2 ạ:\n'},
      {'type': 'link',
       'url': 'https://github.com/tinyvyy/atom-assignments/blob/main/ATOM_Home_Assignment_2.ipynb'}]}]}],
 'thread_ts': '1621044778.004500',
 'reply_count': 2,
 'reply_users_count': 2,
 'latest_reply': '1621167586.003100',
 'reply_users': ['U01V00KHHHP', 'U01UMBZHU2W'],
 'is_locked': False,
 'subscribed': False,
 'reactions': [{'name': 'white_check_mark',
   'users': ['U01V00KHHHP'],
   'count': 1}]}

In [89]:
not_learners_id = ['U01BE2PR6LU']

In [72]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 13:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/hoaintp/
2021-05-03 17:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 03:10:06.001300 https://github.com/Tenderriver/
2021-05-03 08:51:19.001200 U01V00KHHHP 1 ['U01UMBZHU2W'] 2021-05-14 17:51:26.024600 https://github.com/saturn1101/
2021-04-26 06:53:56.000800 U01UMBZHU2W 2 ['U01UMJ451V1', 'U01UMBZHU2W'] 2021-05-14 17:52:35.025100 https://github.com/saturn1101/
2021-04-24 02:01:27.043200 U01VB63LRNC 1 ['U01UMJ451V1'] 2021-05-11 12:37:14.003100 https://github.com/AnhThuNguyenHuynh/


In [134]:
keys = list(map(str, input().rstrip().split()))#user text ts reply_count reply_users_count latest_reply 
#subkeys = list(map(str, input().rstrip().split()))#title phone skype real_name_normalized display_name_normalized

user text ts reply_count reply_users_count latest_reply


In [229]:
def get_info_assign(data,keys,assign_to):
    assign ={}
    assign['assign_num'] = [assign_to]
    for i in keys:
        if i in data.keys() and i =='reply_users':
            assign[i] = data[i]
        elif i in data.keys() and i =='text':
            index1 = data[i].index('<')
            index2 = data[i].index('>')
            github_link = data[i][index1+1:index2]
            assign[i] = [github_link]
        elif i in data.keys():
            assign[i] = [data[i]]
        else:
            assign[i] = ''
    df_assign   = pd.DataFrame(assign)
    return df_assign
        
        
        

In [230]:
list_1 = []
for i in range(len(msg_ls)):
    if msg_ls[i]['user'] != not_learners_id and '://' in msg_ls[i]['text'] :
        text = msg_ls[i]['text']
        index3 = msg_ls[i]['text'].index('://')
        link = text[index3+3:index3+9]
        if link == 'github':
            result = get_info_assign(msg_ls[i],keys,'assignment_2')
            list_1.append(result)
df_assign = pd.concat(list_1,ignore_index=True)
df_assign.head(5)

,assign_num,user,text,ts,reply_count,reply_users_count,latest_reply
0,assignment_2,U01UTGRMGEQ,https://github.com/nguyenhonghanh2511/datacrac...,1621211298.005500,,,
1,assignment_2,U01V00KHHHP,https://github.com/saturn1101/atom-assignments...,1621115007.006100,,,
2,assignment_2,U01UMBZHU2W,https://github.com/tinyvyy/atom-assignments/bl...,1621044778.004500,2,2,1621167586.003100
3,assignment_2,U01UJ9M3P1T,https://github.com/cnhhoang850/atom-assignments,1621044117.003500,,,
4,assignment_2,U01UJ9LG5U5,https://github.com/danhpcv/atom-assignments/tr...,1621043199.002700,,,


In [231]:
df_assign['ts_time'] =df_assign['ts'].apply(lambda x : dt.fromtimestamp(float(x)))
df_assign['latest_reply_time'] =df_assign['latest_reply'].apply(lambda x : 'NULL' if x == '' else
                                                                dt.fromtimestamp(float(x)))
df_assign.drop(['ts', 'latest_reply'], axis='columns', inplace=True)

In [233]:
df_assign['ts_time']= pd.to_datetime(df_assign['ts_time'])
df_assign['latest_reply_time']= df_assign['latest_reply_time'].apply(lambda x : 'NULL' if x == 'NULL'
                                                                    else pd.to_datetime(x))

In [234]:
df_assign.head(5)

,assign_num,user,text,reply_count,reply_users_count,ts_time,latest_reply_time
0,assignment_2,U01UTGRMGEQ,https://github.com/nguyenhonghanh2511/datacrac...,,,2021-05-17 07:28:18.005500,NULL
1,assignment_2,U01V00KHHHP,https://github.com/saturn1101/atom-assignments...,,,2021-05-16 04:43:27.006100,NULL
2,assignment_2,U01UMBZHU2W,https://github.com/tinyvyy/atom-assignments/bl...,2,2,2021-05-15 09:12:58.004500,2021-05-16 19:19:46.003100
3,assignment_2,U01UJ9M3P1T,https://github.com/cnhhoang850/atom-assignments,,,2021-05-15 09:01:57.003500,NULL
4,assignment_2,U01UJ9LG5U5,https://github.com/danhpcv/atom-assignments/tr...,,,2021-05-15 08:46:39.002700,NULL


In [235]:
df_assign['year_submission'] = df_assign.ts_time.dt.year
df_assign['month_submission'] = df_assign.ts_time.dt.month
df_assign['day_submission'] = df_assign.ts_time.dt.day
df_assign['hour_submission'] = df_assign.ts_time.dt.hour

In [257]:
df_assign.head(5)

,assign_num,user,text,reply_count,reply_users_count,ts_time,latest_reply_time,year_submission,month_submission,day_submission,hour_submission
0,assignment_2,U01UTGRMGEQ,https://github.com/nguyenhonghanh2511/datacrac...,,,2021-05-17 07:28:18.005500,NULL,2021,5,17,7
1,assignment_2,U01V00KHHHP,https://github.com/saturn1101/atom-assignments...,,,2021-05-16 04:43:27.006100,NULL,2021,5,16,4
2,assignment_2,U01UMBZHU2W,https://github.com/tinyvyy/atom-assignments/bl...,2,2,2021-05-15 09:12:58.004500,2021-05-16 19:19:46.003100,2021,5,15,9
3,assignment_2,U01UJ9M3P1T,https://github.com/cnhhoang850/atom-assignments,,,2021-05-15 09:01:57.003500,NULL,2021,5,15,9
4,assignment_2,U01UJ9LG5U5,https://github.com/danhpcv/atom-assignments/tr...,,,2021-05-15 08:46:39.002700,NULL,2021,5,15,8


### TODO#4
* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [255]:
sheet_index_no = 1
spreadsheet_key = '1mQJRxLHzDPwpP_BAaCdtQGnP_f9ItcNg_dWaMmKs5eA' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, df_assign) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

#### Link Google Sheet:

https://docs.google.com/spreadsheets/d/1mQJRxLHzDPwpP_BAaCdtQGnP_f9ItcNg_dWaMmKs5eA/edit#gid=0

-------------- HẾT PHẦN OPTION 2 ---------------------